In [34]:
rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr)

In [35]:
##Crear folders de almacenamiento
dir.create("vivienda/microdatos", showWarnings = F)

In [36]:
# Función para descargar y descomprimir un archivo con timeout
download_and_unzip <- function(url, destination="vivienda/microdatos") {
  tryCatch({
    # Descargar el archivo con timeout
    download.file(url, destfile = paste0(destination, "/", basename(url)), mode = "wb", timeout = 10000)

    # Descomprimir el archivo
    unzip(paste0(destination, "/", basename(url)), exdir = destination)

    # Eliminar el archivo comprimido después de descomprimirlo
    unlink(paste0(destination, "/", basename(url)))

    message(paste("Descarga y descompresión exitosas para", basename(url)))
  }, error = function(e) {
    warning(paste("Error al descargar o descomprimir", basename(url), ":", conditionMessage(e)))
  })
}

In [42]:
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enigh/nc/2024/microdatos/enigh2024_ns_concentradohogar_csv.zip")
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enigh/nc/2024/microdatos/enigh2024_ns_viviendas_csv.zip")

Descarga y descompresión exitosas para enigh2024_ns_concentradohogar_csv.zip

Descarga y descompresión exitosas para enigh2024_ns_viviendas_csv.zip



In [43]:
viviendas <- read_csv("vivienda/microdatos/viviendas.csv",show_col_types=FALSE)%>%

mutate(cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es 9
  filter(cve_ent == "09")%>%
  filter(tenencia %in% c(3,4))%>%
  mutate(tenenciac= case_when(
    tenencia %in% c (3,4) ~ "Propia",
    TRUE ~ "Otra"),
    forma_adq=case_when(tipo_adqui %in% c(1) ~ "Compra",
                        tipo_adqui %in% c(2) ~ "Manda construir",
                        tipo_adqui %in% c(3) ~ "Autoconstruye",
                        tipo_adqui>=4 ~ "Otra"))%>%
  select(folioviv,tenencia,tipo_adqui,tenenciac,forma_adq)

head(viviendas,10)
print(unique(viviendas$tipo_adqui))

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"


folioviv,tenencia,tipo_adqui,tenenciac,forma_adq
<chr>,<dbl>,<dbl>,<chr>,<chr>
0903222401,4,1,Propia,Compra
0903222402,4,1,Propia,Compra
0903222405,4,1,Propia,Compra
0903222406,3,1,Propia,Compra
0903223604,4,1,Propia,Compra
0903223605,4,1,Propia,Compra
0903224801,4,2,Propia,Manda construir
0903224802,4,1,Propia,Compra
0903224803,4,2,Propia,Manda construir


[1] 1 2 4 5 3 6


In [44]:
hogares<- read_csv("vivienda/microdatos/concentradohogar.csv",show_col_types=FALSE)%>%
  mutate(cve_ent = case_when(
      nchar(folioviv) == 9  ~ paste0("0", substr(folioviv, 1, 1)),
      nchar(folioviv) == 10 ~ substr(folioviv, 1, 2)
    )) %>%
    # Filtrar si entidad es 9
  filter(cve_ent == "09")%>%
  #Crear deciles
  mutate(decil = ntile(ing_cor, 10))%>%
  #Pegar con viviendas
  left_join(viviendas, by = "folioviv")%>%
    filter(tenenciac == "Propia")

In [45]:
dm<-hogares%>%
as_survey_design(ids = upm, weights = factor,strata=est_dis,
 nest = TRUE)

In [46]:
dm%>%
filter(decil %in%  c(1))%>%
group_by(forma_adq)%>%
summarise(
  hogares=survey_total(vartype="cv"),
  pct=survey_prop( vartype="cv"),
  )%>%
  mutate(pct=round(pct*100,1))%>%
  #Ordenar por pct
  arrange(desc(pct))

forma_adq,hogares,hogares_cv,pct,pct_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Manda construir,33334,0.2545763,39.2,0.1914660
Autoconstruye,19211,0.4342627,22.6,0.3930324
Otra,19097,0.3013859,22.5,0.3017634
Compra,13353,0.4571526,15.7,0.4160678


In [ ]:
39.2+22.6

[1] 61.8